In [1]:
import pandas as pd
from scipy.io import arff
import numpy as np
from scipy import stats
import tensorflow as tf

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef
import erlc
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import time
from tensorflow.keras.utils import to_categorical


## Data
In this section we process and clean the data.

In [2]:
print('Loading dataset #1')
data = pd.DataFrame(arff.loadarff('input/power_multiclass/data1.arff')[0])

for i in range(2,16):
    print('Loading dataset #{}'.format(i))
    dataTemp = pd.DataFrame(arff.loadarff('input/power_multiclass/data{}.arff'.format(i))[0])
    data = pd.concat([data,dataTemp],axis=0)

print("Finished Loading. Final Size = {}".format(data.shape))

Loading dataset #1
Loading dataset #2
Loading dataset #3
Loading dataset #4
Loading dataset #5
Loading dataset #6
Loading dataset #7
Loading dataset #8
Loading dataset #9
Loading dataset #10
Loading dataset #11
Loading dataset #12
Loading dataset #13
Loading dataset #14
Loading dataset #15
Finished Loading. Final Size = (78377, 129)


In [3]:
def dataProc(df):
    '''
    This function takes a dataframe and splits it into data and labels, proccesses them into numpy arrays and
    splits them into training, validation, and testing data and labels.
    '''
    df = df.astype(np.float64)
    label = df['marker'].astype(int)
    df = df.drop(['marker'], axis=1)
    
    df = df.drop(['snort_log1','snort_log2','snort_log3','snort_log4',
                'control_panel_log1','control_panel_log2','control_panel_log3','control_panel_log4',
                'relay1_log','relay2_log','relay3_log','relay4_log'], axis=1)
    
    df = df.reset_index()
    
    df = df.replace(-np.inf, 0)
    df = df.replace(np.inf, 0)

    
    # Converting to arrays
    X = np.asarray(df)
    y =  np.asarray(label)
    
    # Scaling data
    scalar = preprocessing.MinMaxScaler()
    X = scalar.fit_transform(X)
    
    return X,y

In [4]:
X,y = dataProc(data)
print('Size of X: {}'.format(X.shape))
print('Size of y: {}'.format(y.shape))

Size of X: (78377, 117)
Size of y: (78377,)


## Model Testing
This is how we train and test the model.

In [5]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
# model = erlc.ERLC()
# model.fit(X_train, y_train)

In [7]:
# y_pred = model.predict(X_test)
# print(y_pred.shape)

In [8]:
# print(model.__class__.__name__)
# acc = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='weighted')
# mcc = matthews_corrcoef(y_test, y_pred)
# print("Accuracy = {}, F1-score = {}, MCC = {}".format(acc, f1, mcc))

## Cross Validation
For more accurate results, we want to run cross validation on our model, as well as the other models that it's being compared to.

In [9]:
def runCV(model, X, y, numFolds=10):
    cv = KFold(numFolds, True, 1)
    acc = []
    f1 = []
    mcc = []
    trainT = []
    testT = []
    
    name = model.__class__.__name__
    print("Performing CV on {}".format(name))
    i = 1;
    
    for train, test in cv.split(X):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        print("Train shape {}".format(X_train.shape))
        print("Test shape {}".format(X_test.shape))

        time1 = time.time()
        model.fit(X_train, y_train)
        time2 = time.time()
        trainTime= time2-time1
        
        time1 = time.time()
        y_pred = model.predict(X_test)
        time2 = time.time()
        predictTime = time2-time1
        
        acc_temp = accuracy_score(y_test, y_pred)
        f1_temp = f1_score(y_test, y_pred, average='weighted')
        mcc_temp = matthews_corrcoef(y_test, y_pred)
        
        print("Fold #{}".format(i))
        print("Accuracy = {}, F1-score = {}, MCC = {}, Train Time = {}".format(acc_temp, f1_temp, mcc_temp, trainTime))
        
        
        acc.append(acc_temp)
        f1.append(f1_temp)
        mcc.append(mcc_temp)
        trainT.append(trainTime)
        testT.append(predictTime)
        i=i+1
    
    return acc, f1, mcc, trainT, testT

In [10]:
modelDT = DecisionTreeClassifier()
modelRF = RandomForestClassifier()
modelKNN = KNeighborsClassifier()
modelERLC = erlc.ERLC()

In [11]:
MODELS = [
    modelDT, modelRF, modelKNN, 
          modelERLC
]
NUM_FOLDS = 10
writer = pd.ExcelWriter('output/Results_{}CV.xlsx'.format(NUM_FOLDS))

for model in MODELS:
    name = model.__class__.__name__
    acc, f1 , mcc, trainT, testT = runCV(model,X,y, numFolds = NUM_FOLDS)
    
    df = pd.DataFrame({
        'Accuracy': acc,
        'F1-Score': f1,
        'MCC': mcc,
        'Training Time': trainT,
        'Testing Time': testT
    })
    
    
    df.to_excel(writer, name)
    

writer.save()

Performing CV on DecisionTreeClassifier
Train shape (70539, 117)
Test shape (7838, 117)
Fold #1
Accuracy = 0.913498341413626, F1-score = 0.9134247155630819, MCC = 0.9106598417887343, Train Time = 16.258458375930786
Train shape (70539, 117)
Test shape (7838, 117)
Fold #2
Accuracy = 0.9142638428170452, F1-score = 0.914373425597207, MCC = 0.9113872629477601, Train Time = 19.327667713165283
Train shape (70539, 117)
Test shape (7838, 117)
Fold #3
Accuracy = 0.905715743812197, F1-score = 0.9058238254669401, MCC = 0.9025812548792309, Train Time = 9.742313385009766
Train shape (70539, 117)
Test shape (7838, 117)
Fold #4
Accuracy = 0.9117121714723143, F1-score = 0.9117216212363841, MCC = 0.9088228772719543, Train Time = 9.502947568893433
Train shape (70539, 117)
Test shape (7838, 117)
Fold #5
Accuracy = 0.9112018372033682, F1-score = 0.9113060447359627, MCC = 0.9083031126183285, Train Time = 9.64749002456665
Train shape (70539, 117)
Test shape (7838, 117)
Fold #6
Accuracy = 0.9089053329931105, 

221/221 [==============================] - 4s 16ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 22/500
221/221 [==============================] - 6s 29ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 23/500
221/221 [==============================] - 4s 20ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 24/500
221/221 [==============================] - 4s 18ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 25/500
221/221 [==============================] - 4s 18ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 26/500
221/221 [==============================] - 4s 17ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 27/500
221/221 [==============================] - 6s 28ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 28/500
221/221 [==============================] - 4s 20ms/step

Epoch 82/500
221/221 [==============================] - 7s 30ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 83/500
221/221 [==============================] - 5s 21ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 84/500
221/221 [==============================] - 5s 21ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 85/500
221/221 [==============================] - 4s 19ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 86/500
221/221 [==============================] - 4s 18ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 87/500
221/221 [==============================] - 6s 29ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 88/500
221/221 [==============================] - 5s 23ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 89/500
221/221 [==============================] -

552/552 [==============================] - 3s 6ms/step - loss: 0.5059 - acc: 0.8064 - f1_m: 0.8025
Epoch 40/500
552/552 [==============================] - 3s 6ms/step - loss: 0.4880 - acc: 0.8151 - f1_m: 0.8109
Epoch 41/500
552/552 [==============================] - 3s 6ms/step - loss: 0.4590 - acc: 0.8264 - f1_m: 0.8231
Epoch 42/500
552/552 [==============================] - 3s 6ms/step - loss: 0.4636 - acc: 0.8232 - f1_m: 0.8213
Epoch 43/500
552/552 [==============================] - 3s 6ms/step - loss: 0.4467 - acc: 0.8303 - f1_m: 0.8269
Epoch 44/500
552/552 [==============================] - 3s 6ms/step - loss: 0.4567 - acc: 0.8275 - f1_m: 0.8246
Epoch 45/500
552/552 [==============================] - 3s 6ms/step - loss: 0.4488 - acc: 0.8322 - f1_m: 0.8296
Epoch 46/500
552/552 [==============================] - 3s 6ms/step - loss: 0.4055 - acc: 0.8473 - f1_m: 0.8452
Epoch 47/500
552/552 [==============================] - 3s 6ms/step - loss: 0.4025 - acc: 0.8488 - f1_m: 0.8460
Epoch

552/552 [==============================] - 4s 7ms/step - loss: 0.4710 - acc: 0.8199 - f1_m: 0.8158
Epoch 25/500
552/552 [==============================] - 4s 7ms/step - loss: 0.4305 - acc: 0.8378 - f1_m: 0.8346
Epoch 26/500
552/552 [==============================] - 4s 7ms/step - loss: 0.4050 - acc: 0.8477 - f1_m: 0.8463
Epoch 27/500
552/552 [==============================] - 4s 7ms/step - loss: 0.3858 - acc: 0.8545 - f1_m: 0.8517
Epoch 28/500
552/552 [==============================] - 4s 7ms/step - loss: 0.3562 - acc: 0.8672 - f1_m: 0.8658
Epoch 29/500
552/552 [==============================] - 4s 7ms/step - loss: 0.3496 - acc: 0.8701 - f1_m: 0.8687
Epoch 30/500
552/552 [==============================] - 4s 7ms/step - loss: 0.3276 - acc: 0.8786 - f1_m: 0.8783
Epoch 31/500
552/552 [==============================] - 4s 7ms/step - loss: 0.3232 - acc: 0.8806 - f1_m: 0.8796
Epoch 32/500
552/552 [==============================] - 4s 7ms/step - loss: 0.2912 - acc: 0.8914 - f1_m: 0.8912
Epoch

...

221/221 [==============================] - 4s 19ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0029 - val_mse: 0.0029
Epoch 57/500
221/221 [==============================] - 4s 19ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0029 - val_mse: 0.0029
Epoch 58/500
221/221 [==============================] - 4s 18ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0030 - val_mse: 0.0030
Epoch 59/500
221/221 [==============================] - 4s 17ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0030 - val_mse: 0.0030
Epoch 60/500
221/221 [==============================] - 6s 27ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0030 - val_mse: 0.0030
Epoch 61/500
221/221 [==============================] - 4s 19ms/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.0030 - val_mse: 0.0030
Epoch 62/500
221/221 [==============================] - 4s 19ms/step - loss: 0.0029 - mse: 0.0029 - val_loss: 0.0030 - val_mse: 0.0030
Epoch 63/500
221/221 [==============================] - 4s 18ms/step

552/552 [==============================] - 8s 14ms/step - loss: 1.1499 - acc: 0.5475 - f1_m: 0.4799
Epoch 17/500
552/552 [==============================] - 5s 10ms/step - loss: 1.1184 - acc: 0.5583 - f1_m: 0.4980
Epoch 18/500
552/552 [==============================] - 5s 9ms/step - loss: 1.0691 - acc: 0.5771 - f1_m: 0.5256
Epoch 19/500
552/552 [==============================] - 5s 9ms/step - loss: 1.0536 - acc: 0.5828 - f1_m: 0.5334
Epoch 20/500
552/552 [==============================] - 7s 13ms/step - loss: 1.0064 - acc: 0.6023 - f1_m: 0.5591
Epoch 21/500
552/552 [==============================] - 6s 10ms/step - loss: 0.9870 - acc: 0.6141 - f1_m: 0.5714
Epoch 22/500
552/552 [==============================] - 5s 9ms/step - loss: 0.9644 - acc: 0.6212 - f1_m: 0.5844
Epoch 23/500
552/552 [==============================] - 5s 9ms/step - loss: 0.9349 - acc: 0.6368 - f1_m: 0.6000
Epoch 24/500
552/552 [==============================] - 4s 8ms/step - loss: 0.8721 - acc: 0.6611 - f1_m: 0.6312
E

552/552 [==============================] - 5s 8ms/step - loss: 0.2313 - acc: 0.9150 - f1_m: 0.9154
Epoch 89/500
552/552 [==============================] - 5s 10ms/step - loss: 0.2029 - acc: 0.9255 - f1_m: 0.9262
Epoch 90/500
552/552 [==============================] - 7s 12ms/step - loss: 0.2880 - acc: 0.9015 - f1_m: 0.9018
Epoch 91/500
552/552 [==============================] - 6s 10ms/step - loss: 0.1776 - acc: 0.9366 - f1_m: 0.9367
Epoch 92/500
552/552 [==============================] - 5s 9ms/step - loss: 0.2062 - acc: 0.9253 - f1_m: 0.9256
Epoch 93/500
552/552 [==============================] - 5s 8ms/step - loss: 0.2134 - acc: 0.9232 - f1_m: 0.9233
Epoch 94/500
552/552 [==============================] - 8s 14ms/step - loss: 0.2105 - acc: 0.9241 - f1_m: 0.9244
Epoch 95/500
552/552 [==============================] - 5s 10ms/step - loss: 0.2054 - acc: 0.9257 - f1_m: 0.9260
Epoch 96/500
552/552 [==============================] - 5s 9ms/step - loss: 0.2375 - acc: 0.9158 - f1_m: 0.9159


552/552 [==============================] - 6s 11ms/step - loss: 0.1724 - acc: 0.9388 - f1_m: 0.9386
Epoch 60/500
552/552 [==============================] - 6s 10ms/step - loss: 0.1506 - acc: 0.9464 - f1_m: 0.9466
Epoch 61/500
552/552 [==============================] - 5s 10ms/step - loss: 0.1681 - acc: 0.9396 - f1_m: 0.9394
Epoch 62/500
552/552 [==============================] - 7s 14ms/step - loss: 0.1717 - acc: 0.9384 - f1_m: 0.9385
Epoch 63/500
552/552 [==============================] - 6s 11ms/step - loss: 0.1340 - acc: 0.9525 - f1_m: 0.9525
Epoch 64/500
552/552 [==============================] - 6s 10ms/step - loss: 0.1677 - acc: 0.9398 - f1_m: 0.9400
Epoch 65/500
552/552 [==============================] - 5s 10ms/step - loss: 0.1565 - acc: 0.9438 - f1_m: 0.9438
Epoch 66/500
552/552 [==============================] - 4s 7ms/step - loss: 0.1349 - acc: 0.9525 - f1_m: 0.9525
Epoch 67/500
552/552 [==============================] - 8s 14ms/step - loss: 0.1614 - acc: 0.9426 - f1_m: 0.94

441/441 [==============================] - 2s 5ms/step - loss: 5.8267e-08 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 6.8663e-08 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 25/500
441/441 [==============================] - 2s 4ms/step - loss: 4.9328e-08 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 5.8692e-08 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 26/500
441/441 [==============================] - 2s 4ms/step - loss: 4.1539e-08 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.9499e-08 - val_acc: 1.0000 - val_f1_m: 1.0000
Epoch 27/500
441/441 [==============================] - 2s 4ms/step - loss: 3.4295e-08 - acc: 1.0000 - f1_m: 1.0000 - val_loss: 4.1750e-08 - val_acc: 1.0000 - val_f1_m: 1.0000
Training complete
Fold #10
Accuracy = 0.9502360597167283, F1-score = 0.9501720035542908, MCC = 0.9485872244694172, Train Time = 1763.2135226726532
